<a href="https://www.kaggle.com/code/aisuko/text-generation-with-outlines?scriptVersionId=164531661" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# Outlines

Neuro-sysmbolic Text Generation

It is a library for neural text generation and it can be a replacement for the `generate` method in the transformers library. And it can guarantee that the output will match a regular expressions or follow a JSON schema.

# Installation

In [1]:
!pip install outlines==0.0.9

  Obtaining dependency information for outlines==0.0.9 from https://files.pythonhosted.org/packages/79/db/fbee799e6c6a602b654ec87ee3eb2973a215417a83d3265e060a177a2508/outlines-0.0.9-py3-none-any.whl.metadata
  Obtaining dependency information for interegular from https://files.pythonhosted.org/packages/c4/01/72d6472f80651673716d1deda2a5bbb633e563ecf94f4479da5519d69d25/interegular-0.3.3-py37-none-any.whl.metadata
  Obtaining dependency information for lark from https://files.pythonhosted.org/packages/e7/9c/eef7c591e6dc952f3636cfe0df712c0f9916cedf317810a3bb53ccb65cdd/lark-1.1.9-py3-none-any.whl.metadata
  Obtaining dependency information for perscache from https://files.pythonhosted.org/packages/76/b2/596bd2fc57f0031486cb240da527c993d9bdf1fc51990930cb9bd554a4ed/perscache-0.6.1-py2.py3-none-any.whl.metadata
  Obtaining dependency information for pydantic>=2.0 from https://files.pythonhosted.org/packages/42/28/f19be8d493c59a8ddf32c15d69765c6423ad712da1c6255b418df2fc1443/pydantic-2.6.2-py3-

# Early stopping

Stoping the generation after a given sequence has been found.

In [2]:
import outlines.text.generate as generate
import outlines.models as models

model = models.transformers("gpt2")

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [3]:
answer = generate.continuation(model, stop=["."])("Tell me a one-sentence joke.")
print(answer)

 I don't ever pay attention to this show


# Multiple choices

You can reduce the completion to a choice between multiple possibilities.

In [4]:
prompt="""You care a sentiment-labelling assistant. Is the following review positive or negative? Review: This restaurant is just awesome!"""
answer =generate.choice(model, ["Positive", "Negative"])(prompt)
print(answer)

/opt/conda/lib/python3.10/site-packages/outlines/text/fsm.py:604: NumbaPendingDeprecationWarning: 
Encountered the use of a type that is scheduled for deprecation: type 'reflected set' found for argument 'fsm_finals' of function '_walk_fsm'.

For more information visit https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-reflection-for-list-and-set-types

File "../../opt/conda/lib/python3.10/site-packages/outlines/text/fsm.py", line 239:
@numba.njit(nogil=True, cache=True)
def _walk_fsm(
^

  state_seq = _walk_fsm(
/opt/conda/lib/python3.10/site-packages/numba/core/ir_utils.py:2149: NumbaPendingDeprecationWarning: 
Encountered the use of a type that is scheduled for deprecation: type 'reflected set' found for argument 'fsm_finals' of function 'state_scan_tokens'.

For more information visit https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-reflection-for-list-and-set-types

File "../../opt/conda/lib/python3.10/site-packages/out

Negative


# Efficient JSON generation following a Pydantic model

Allowing to guide the generation process so the output is guaranteed to follow a JSON schema or Pydantic model

In [5]:
from enum import Enum
from pydantic import BaseModel, constr
import torch

class Weapon(str, Enum):
    sword="sword"
    axe="axe"
    mace="mace"
    spear="spear"
    bow="bow"
    crossbow="crossbow"

class Armor(str, Enum):
    leather="leather"
    chainmail="chainmail"
    plate="plate"

class Character(BaseModel):
    name: constr(max_length=10)
    age: int
    armor: Armor
    weapon: Weapon
    strength: int

model = models.transformers("gpt2", device="cuda")

# Construct guided sequence generator
generator = generate.json(model, Character, max_tokens=100)

# Draw a sample
rng = torch.Generator(device="cuda")
rng.manual_seed(789001)

In [6]:
sequence = generator("Give me a character description", rng=rng)
print(sequence)

{ "name":"Janston", "age": 3, "armor" : "chainmail", "weapon" : "bow", "strength" : 30 }


In [7]:
sequence = generator("Give me an interesting character description", rng=rng)
print(sequence)

{

"name" : "version 1,",

"age" : 0,

"armor" : "plate",

"weapon" : "spear",

"strength" : 0

}


In [8]:
parsed = Character.model_validate_json(sequence)
print(parsed)

name='version 1,' age=0 armor=<Armor.plate: 'plate'> weapon=<Weapon.spear: 'spear'> strength=0


# Prompting

Managing prompts by encapsulating templates inside "template functions". Template functions require no superfluous abstraction, they use the Jinja2 templating engine to help build complex prompts in a concise manner

In [9]:
import outlines.text as text

examples=[
    ("The food was digusting","Negative"),
    ("We had a fantastic night", "Positive"),
    ("Recommended","Positive"),
    ("The waiter was rude","Negative")
]

@text.prompt
def labelling(to_label, examples):
    """You are a sentiment-labelling assistant.
    {% for example in examples %}
    {{ example[0] }} // {{ example[1] }}
    {% endfor %}
    {{ to_label }} //
    """

prompt=labelling("Just awesome", examples)
answer = text.generate.continuation(model, max_tokens=100)(prompt)
print(answer)

 Check out more at Food Standards in the comments
I checked it out when I came and quite enjoyed it down the hall and it was addictive and on top of that it was delicious. Everything was done especially the broccolini with the green zest. I highly recommend this dish!
[^I trusted my teacher if I can´t wipe the toilet].
Chicken picture mab was fun // Yahtzee
This was really good // Neptunes
~~~~~~~~ //thinks it was sloppy
